In [ ]:
# Imports
import numpy as np
import os
import pickle
import requests
import random
import shutil
import folium

from shapely.geometry import box

In [ ]:
# Configuration of CONSTANTS

NRW_Center_Coordinates = [51.245649, 6.824185]

# Specify directory into which you want to download the images
TARGET_DIR = "/Users/Kevin/Desktop/NRW_Test"

# How many images do you want to download?
DOWNLOAD_SIZE = 10

# How large is the bounding box around the coordinate (in degrees)
NORTH_SOUTH_SPAN = 1.0
EAST_WEST_SPAN = 1.2

# URL dummy for image request to openNRW server
WMS_1 = 'https://www.wms.nrw.de/geobasis/wms_nw_dop?SERVICE=WMS&REQUEST=GetMap&Version=1.1.1&LAYERS=nw_dop_rgb&SRS=EPSG:4326&BBOX='
WMS_2 = '&WIDTH=320&HEIGHT=320&FORMAT=image/png;%20mode=8bit'

# Avg. earth radius in meters
R = 6371000

# Image side length in meters
SIDE = 16

# Distance in north-south direction
DLAT = (SIDE * 360) / (2 * np.pi * R)

In [ ]:
def download_image(lon, lat):

    """
    The function parameters lon and lat specify the center of a given image.
    However, the openNRW serves expects the bounding box coordinates for a given image.
    Hence, we convert the lon and lat to x_min, x_max, y_min, and y_max. 
    """
    y_max = lat + dlat/2
    x_min = lon - (((side * 360) / (2 * np.pi * r * np.cos(np.deg2rad(y_max))))/2)
    y_min = lat - dlat/2
    x_max = lon + (((side * 360) / (2 * np.pi * r * np.cos(np.deg2rad(y_max))))/2)

    
    # Specify URL from which we download our tile
    url = os.path.join(WMS_1 + str(x_min) + ',' + str(y_min) + ',' + str(x_max) + ',' + str(y_max) + WMS_2)

    # Download tile imagery from URL
    response = requests.get(url, stream=True)
    
    # Save downloaded file under current_save_path
    with open(os.path.join(TARGET_DIR, str(lat) + ',' + str(lon) + ".png"), 'wb') as out_file:

        response.raw.decode_content = True
        shutil.copyfileobj(response.raw, out_file)

    del response

In [ ]:
# Center point coordinates of your chosen location
lat_center = NRW_Center_Coordinates[0]
lon_center = NRW_Center_Coordinates[1]

# Create bounding box over chosen location
rec_south = lat_center - (NORTH_SOUTH_SPAN/2)
rec_north = lat_center + (NORTH_SOUTH_SPAN/2)
rec_east = lon_center + (EAST_WEST_SPAN/2)
rec_west = lon_center - (EAST_WEST_SPAN/2)

upper_left = (rec_north, rec_west)
upper_right = (rec_north, rec_east)
lower_left = (rec_south, rec_west)
lower_right = (rec_south, rec_east)

# Create a map to visualize center point coordinates and the respective bounding box 
m = folium.Map(location=NRW_Center_Coordinates, zoom_start=8)
# Add center point to map
folium.Marker(location=[lat_center, lon_center]).add_to(m)
# Add bounding box to map
folium.Rectangle(bounds=[upper_left, upper_right, lower_right, lower_left], color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(m)
# Display map
m

In [ ]:
for i in range(0, DOWNLOAD_SIZE):
    
    # Select random coordinates within the specified bounding box
    lat = rec_south + (rec_north - rec_south) * random.random()
    lon = rec_west + (rec_east - rec_west) * random.random()

    # Download image with respective coordinates
    download_image(lon, lat)  